# Lending Club Part 1 - Ingestion, Cleaning and Pre Processing

In [4]:
import pandas as pd
import os
from sys import platform
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pickle
import seaborn

In [5]:
def is_integer(x):
    '''
    This function returns True if x is an integer, and False otherwise
    '''
    try:
        return (int(x) == float(x))
    except:
        return False

## Ingestion

Function ingests files in specified directory into a pandas dataframe. It will return a dictionary containing these dataframes, keyed by the file name. We assume the directory is a replication of 'Lending Club Data->2016' directory on Canvas

In [6]:
dir_2016 = "./data/2016/securev/"
pickle_file = "./data/pickle/clean_data.pickle"

In [7]:
def ingest_files(directory):
    # If the directory has no trailing slash, add one
    if directory[-1] != "/":
        directory = directory + "/"
    all_files = os.listdir(directory)
    output = {}
    
    print("Directory " + directory + " has " + str(len(all_files)) + " files:")
    for i in all_files:
        print("    Reading file " + i)
        output[i] = pd.read_csv(directory + i, dtype = str, skiprows = 1)     
        # Remove lines with non-integer IDs
        invalid_rows = (output[i].id.apply( lambda x : is_integer(x) == False ))
        if invalid_rows.sum() > 0:
            print("        Found " + str(invalid_rows.sum()) + " invalid rows which were removed")
            output[i] = output[i][invalid_rows == False]
    return output

files_2016 = ingest_files(dir_2016)
data_2016 = pd.concat(files_2016.values()).reset_index(drop = True)

Directory ./data/2016/securev/ has 4 files:
    Reading file LoanStats_securev1_2016Q2.csv
        Found 2 invalid rows which were removed
    Reading file LoanStats_securev1_2016Q3.csv
        Found 2 invalid rows which were removed
    Reading file LoanStats_securev1_2016Q1.csv
        Found 2 invalid rows which were removed
    Reading file LoanStats_securev1_2016Q4.csv
        Found 2 invalid rows which were removed


## Pre Process

### Create new data frame including only columns of interest

In [9]:
# Identify the columns we'll be keeping from the dataset
cols_to_pick = ['id','loan_amnt','funded_amnt','term','int_rate',
                 'installment','grade','emp_length', 'home_ownership',
                 'annual_inc','verification_status','issue_d',
                 'loan_status','purpose','dti', 'delinq_2yrs',
                 'earliest_cr_line','open_acc','pub_rec', 'fico_range_high',
                 'fico_range_low', 'revol_bal','revol_util', 'total_pymnt',
                                                    'last_pymnt_d', 'recoveries']
final_data = data_2016[cols_to_pick].copy()
print("Starting with " + str(len(final_data)) + " rows")

Starting with 434407 rows


### Typecast the columns

In [10]:
# Identify the type of each of these column
float_cols = ['loan_amnt', 'funded_amnt', 'installment', 'annual_inc',
                     'dti', 'revol_bal', 'delinq_2yrs', 'open_acc', 'pub_rec',
                                'fico_range_high', 'fico_range_low', 'total_pymnt', 'recoveries']
cat_cols = ['term', 'grade', 'emp_length', 'home_ownership',
                    'verification_status', 'loan_status', 'purpose']
perc_cols = ['int_rate', 'revol_util']
date_cols = ['issue_d', 'earliest_cr_line', 'last_pymnt_d']
# Ensure that we have types for every column
assert set(cols_to_pick) - set(float_cols) - set(cat_cols) - set(perc_cols) - set(date_cols) == set(["id"])
# All categorical columns other than "loan_status" will be used as discrete features
discrete_features = list(set(cat_cols) - set(["loan_status"]))
# All numeric columns will be used as continuous features
continuous_features = list(float_cols + perc_cols)

In [11]:
for i in float_cols:
    final_data[i] = final_data[i].astype(float)
    
def clean_perc(x):
    if pd.isnull(x):
        return np.nan
    else:
        return float(x.strip()[:-1])
for i in perc_cols:
    final_data[i] = final_data[i].apply( clean_perc )
    
def clean_date(x):
    if pd.isnull(x):
        return None
    else:
        return datetime.datetime.strptime( x, "%b-%Y").date()
for i in date_cols:
    final_data[i] = final_data[i].apply( clean_date )
    
for i in cat_cols:
    final_data.loc[final_data[i].isnull(), i] = None

## Clean Data

### Loans payed back

Remove all rows for loans that were paid back on the days they were issued and report how many rows were removed

In [ ]:
###############################
###   Write your code  here ##
##############################
    

#############################

In [12]:
final_data['loan_length'] = (final_data.last_pymnt_d - final_data.issue_d) / np.timedelta64(1, 'M')
n_rows = len(final_data)
final_data = final_data[final_data.loan_length != 0]
print("Removed " + str(n_rows - len(final_data)) + " rows")

Removed 2553 rows


### Handle outliers

Write a script that removes rows where annual_inc>10999200 and revol_util>300, report how many rows were removed


In [ ]:
###############################
###   Write your code  here ##
##############################
    

#############################

Write a script that removes loans that are too recent to have been paid off or defaulted, report how many rows were removed

In [ ]:
###############################
###   Write your code  here ##
##############################
    

#############################

In [16]:
# There are quite a few outliers, but the two most obvious
# ones to remove are in annual_inc, revol_util Remove these.
n_rows = len(final_data)
final_data = final_data[final_data.annual_inc < 10999200]
final_data = final_data[final_data.revol_util < 300]
print("Removed " + str(n_rows - len(final_data)) + " rows")


Removed 0 rows


In [15]:
# Remove all loans that are too recent to have been paid off or
# defaulted
n_rows = len(final_data)
final_data = final_data[final_data.loan_status.isin(['Fully Paid','Charged Off','Default'])]
print("Removed " + str(n_rows - len(final_data)) + " rows")


Removed 0 rows


### Drop null values

Dealing with null values, we allow categorical variables to be null OTHER than grade, which is a particularly important category. All non-categorical variables must be non-null, and we drop rows that do not meet this requirement


In [17]:
# Deal with null values. We allow cateogrical variables to be null
# OTHER than grade, which is a particularly important categorical.
# All non-categorical variables must be non-null, and we drop
# rows that do not meet this requirement
required_cols = set(cols_to_pick) - set(cat_cols) - set(["id"])
required_cols.add("grade")

n_rows = len(final_data)
final_data.dropna(subset = required_cols ,inplace=True)
print("Removed " + str(n_rows - len(final_data)) + " rows")


Removed 547 rows


## Calculate returns for each loan

In [18]:
# Define the names of the four returns we'll be calculating
ret_cols = ["ret_PESS", "ret_OPT",'ret_REAL1','ret_REAL2','ret_REAL3']

#### Return Method 2 (pessimistic)
Calculate the return using a simple annualized profit margin
Pessimistic definition (method 2)


In [19]:
final_data['term_num'] = final_data.term.str.extract('(\d+)',expand=False).astype(int)
final_data['ret_PESS'] = ( (final_data.total_pymnt - final_data.funded_amnt) 
                                            / final_data.funded_amnt ) * (12 / final_data['term_num'])


#### Return Method 1 (optimistic)
Assuming that if a loan gives a positive return, we can immediately find a similar loan to invest in; if the loan takes a loss, we use method 2 to compute the return


In [20]:
final_data['ret_OPT'] = ( (final_data.total_pymnt - final_data.funded_amnt)
                                            / final_data.funded_amnt ) * (12 / final_data['loan_length'])
final_data.loc[final_data.ret_OPT < 0,'ret_OPT'] = final_data.ret_PESS[final_data.ret_OPT < 0]


#### Return Method 3 (realistic)
Make a realistic assmption on returns for loans returned early and implement the assumption in code

In [ ]:
###############################
###   Write your code  here ##
##############################
final_data['ret_REAL1'] =
    

#############################

#### Return Method 4 (realistic)
Make another realistic assmption on returns for loans returned early and implement the assumption in code

In [ ]:
###############################
###   Write your code  here ##
##############################
final_data['ret_REAL2'] =
    

#############################

#### Return Method 4 (realistic)
Make a third realistic assmption on returns for loans returned early and implement the assumption in code

In [ ]:
###############################
###   Write your code  here ##
##############################
final_data['ret_REAL3'] =
    

#############################

## Data Exploration
Find the percentage of loans by grade, the default by grade, and the return of each grade


In [21]:
perc_by_grade = (final_data.grade.value_counts()*100/len(final_data)).sort_index()
default_by_grade = final_data.groupby("grade").apply(lambda x : (x.loan_status != "Fully Paid").sum()*100/len(x) )
ret_by_grade_OPT = final_data.groupby("grade").apply(lambda x : np.mean(x.ret_OPT)*100 )
ret_by_grade_PESS = final_data.groupby("grade").apply(lambda x : np.mean(x.ret_PESS)*100 )
int_rate_by_grade = final_data.groupby("grade").apply(lambda x : np.mean(x.int_rate))

combined = pd.DataFrame(perc_by_grade)
combined['default'] = default_by_grade
combined['int_rate'] = int_rate_by_grade
combined['return_OPT'] = ret_by_grade_OPT
combined['return_PESS'] = ret_by_grade_PESS
combined['ret_REAL1'] = ret_by_grade_PESS
combined['ret_REAL2'] = ret_by_grade_PESS
combined['ret_REAL3'] = ret_by_grade_PESS

combined


,grade,default,int_rate,return_OPT,return_PESS
A,15.263089,8.466036,6.877235,3.493205,0.754511
B,29.426487,17.974932,10.247267,3.939571,-0.137705
C,30.199839,27.982891,13.793047,4.054359,-1.334874
D,14.748937,37.896860,18.243862,4.185277,-2.486445
E,6.903233,46.035947,22.378225,4.604398,-3.264838
F,2.693136,54.433575,25.807916,3.746478,-4.352079
G,0.765279,58.911716,28.779178,3.406091,-4.889995


## Save a Pickle
Remove the total_pymnt from the list of continuous features; this variable is highly predictive of the outcome but is not known at the time the loan is issued

In [22]:
continuous_features = [i for i in continuous_features if i not in ["total_pymnt", "recoveries"]]


In [23]:
pickle.dump( [final_data, discrete_features, continuous_features, ret_cols], open(pickle_file, "wb") )